In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2
import numpy as np
import os

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length > frame_count:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

    index = 0
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if index < message_length:
            binary_pixel = '{:08b}'.format(frame[0, 0, 0])
            frame[0, 0, 0] = int(binary_pixel[:-1] + binary_message[index], 2)
            index += 1

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    binary_message = ""
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        binary_pixel = '{:08b}'.format(frame[0, 0, 0])
        binary_message += binary_pixel[-1]

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    cap.release()
    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_type = tk.StringVar()
        self.encode_type.set("Image")

        self.encode_frame = tk.Frame(self.master)
        self.encode_frame.pack()

        self.encode_label = tk.Label(self.encode_frame, text="Select Type:")
        self.encode_label.pack(side="left")

        self.image_radio = tk.Radiobutton(self.encode_frame, text="Image", variable=self.encode_type, value="Image")
        self.image_radio.pack(side="left")

        self.video_radio = tk.Radiobutton(self.encode_frame, text="Video", variable=self.encode_type, value="Video")
        self.video_radio.pack(side="left")

        self.select_button = tk.Button(self.encode_frame, text="Select File", command=self.select_file)
        self.select_button.pack()

        self.hide_frame = tk.Frame(self.master)
        self.hide_frame.pack()

        self.message_label = tk.Label(self.hide_frame, text="Enter Message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.hide_frame, width=50)
        self.message_entry.pack(pady=5)

        self.encode_button = tk.Button(self.hide_frame, text="Hide Message", command=self.hide_message)
        self.encode_button.pack()

        self.decode_frame = tk.Frame(self.master)
        self.decode_frame.pack()

        self.decode_label = tk.Label(self.decode_frame, text="Decoded Message:")
        self.decode_label.pack()

        self.decoded_message = tk.StringVar()
        self.decoded_message.set("")

        self.decoded_message_label = tk.Label(self.decode_frame, textvariable=self.decoded_message)
        self.decoded_message_label.pack()

    def select_file(self):
        file_path = filedialog.askopenfilename()
        self.file_path = file_path

    def hide_message(self):
        message = self.message_entry.get()

        if not message:
            messagebox.showerror("Error", "Please enter a message.")
            return

        if self.encode_type.get() == "Image":
            if self.file_path:
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.png"
                hide_message_in_image(self.file_path, message, output_path)
            else:
                messagebox.showerror("Error", "Please select an image file.")
        else:
            if self.file_path:
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.mp4"
                hide_message_in_video(self.file_path, message, output_path)
            else:
                messagebox.showerror("Error", "Please select a video file.")

    def decode_message(self):
        if self.file_path:
            if self.encode_type.get() == "Image":
                message = decode_message_in_image(self.file_path)
            else:
                message = decode_message_in_video(self.file_path)
            self.decoded_message.set(message)
        else:
            messagebox.showerror("Error", "Please select a file to decode.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2
import numpy as np
import os

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length > frame_count:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

    index = 0
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if index < message_length:
            binary_pixel = '{:08b}'.format(frame[0, 0, 0])
            frame[0, 0, 0] = int(binary_pixel[:-1] + binary_message[index], 2)
            index += 1

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    binary_message = ""
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        binary_pixel = '{:08b}'.format(frame[0, 0, 0])
        binary_message += binary_pixel[-1]

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    cap.release()
    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_or_decode = tk.StringVar()
        self.encode_or_decode.set("Encode")

        self.mode_frame = tk.Frame(self.master)
        self.mode_frame.pack()

        self.encode_radio = tk.Radiobutton(self.mode_frame, text="Encode", variable=self.encode_or_decode, value="Encode")
        self.encode_radio.pack(side="left")

        self.decode_radio = tk.Radiobutton(self.mode_frame, text="Decode", variable=self.encode_or_decode, value="Decode")
        self.decode_radio.pack(side="left")

        self.select_button = tk.Button(self.master, text="Select File", command=self.select_file)
        self.select_button.pack()

        self.message_frame = tk.Frame(self.master)
        self.message_frame.pack()

        self.message_label = tk.Label(self.message_frame, text="Enter Message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.message_frame, width=50)
        self.message_entry.pack(pady=5)

        self.action_button = tk.Button(self.master, textvariable=self.encode_or_decode, command=self.perform_action)
        self.action_button.pack()

        self.result_frame = tk.Frame(self.master)
        self.result_frame.pack()

        self.result_label = tk.Label(self.result_frame, text="")
        self.result_label.pack()

    def select_file(self):
        file_path = filedialog.askopenfilename()
        self.file_path = file_path

    def perform_action(self):
        message = self.message_entry.get()

        if not message:
            messagebox.showerror("Error", "Please enter a message.")
            return

        if self.encode_or_decode.get() == "Encode":
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to encode.")
                return
            if os.path.splitext(self.file_path)[1].lower() in ['.png', '.jpg', '.jpeg']:
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.png"
                hide_message_in_image(self.file_path, message, output_path)
            elif os.path.splitext(self.file_path)[1].lower() in ['.mp4']:
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.mp4"
                hide_message_in_video(self.file_path, message, output_path)
            else:
                messagebox.showerror("Error", "Unsupported file format.")
        else:
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to decode.")
                return
            if os.path.splitext(self.file_path)[1].lower() in ['.png', '.jpg', '.jpeg']:
                message = decode_message_in_image(self.file_path)
            elif os.path.splitext(self.file_path)[1].lower() in ['.mp4']:
                message = decode_message_in_video(self.file_path)
            else:
                messagebox.showerror("Error", "Unsupported file format.")

            self.result_label.config(text="Decoded Message: " + message)

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [40]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2
import numpy as np
import os

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length > frame_count:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

    index = 0
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if index < message_length:
            binary_pixel = '{:08b}'.format(frame[0, 0, 0])
            frame[0, 0, 0] = int(binary_pixel[:-1] + binary_message[index], 2)
            index += 1

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    binary_message = ""
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        binary_pixel = '{:08b}'.format(frame[0, 0, 0])
        binary_message += binary_pixel[-1]

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    cap.release()
    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_or_decode = tk.StringVar()
        self.encode_or_decode.set("Encode")

        self.mode_frame = tk.Frame(self.master)
        self.mode_frame.pack()

        self.encode_radio = tk.Radiobutton(self.mode_frame, text="Encode", variable=self.encode_or_decode, value="Encode", command=self.show_encode_options)
        self.encode_radio.pack(side="left")

        self.decode_radio = tk.Radiobutton(self.mode_frame, text="Decode", variable=self.encode_or_decode, value="Decode", command=self.show_decode_options)
        self.decode_radio.pack(side="left")

        self.file_type = tk.StringVar()
        self.file_type.set("Image")

        self.encode_options_frame = tk.Frame(self.master)
        self.encode_options_frame.pack()

        self.image_radio = tk.Radiobutton(self.encode_options_frame, text="Image", variable=self.file_type, value="Image")
        self.image_radio.pack(side="left")

        self.video_radio = tk.Radiobutton(self.encode_options_frame, text="Video", variable=self.file_type, value="Video")
        self.video_radio.pack(side="left")

        self.select_button = tk.Button(self.master, text="Select File", command=self.select_file)
        self.select_button.pack()

        self.message_frame = tk.Frame(self.master)
        self.message_frame.pack()

        self.message_label = tk.Label(self.message_frame, text="Enter Message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.message_frame, width=50)
        self.message_entry.pack(pady=5)

        self.action_button = tk.Button(self.master, textvariable=self.encode_or_decode, command=self.perform_action)
        self.action_button.pack()

        self.result_frame = tk.Frame(self.master)
        self.result_frame.pack()

        self.result_label = tk.Label(self.result_frame, text="")
        self.result_label.pack()

    def show_encode_options(self):
        if self.encode_or_decode.get() == "Encode":
            self.encode_options_frame.pack()

    def show_decode_options(self):
        if self.encode_or_decode.get() == "Decode":
            self.encode_options_frame.pack_forget()

    def select_file(self):
        file_path = filedialog.askopenfilename()
        self.file_path = file_path

    def perform_action(self):
        message = self.message_entry.get()

        if not message:
            messagebox.showerror("Error", "Please enter a message.")
            return

        if self.encode_or_decode.get() == "Encode":
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to encode.")
                return
            if self.file_type.get() == "Image":
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.png"
                hide_message_in_image(self.file_path, message, output_path)
            elif self.file_type.get() == "Video":
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.mp4"
                hide_message_in_video(self.file_path, message, output_path)
            else:
                messagebox.showerror("Error", "Unsupported file type.")
        else:
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to decode.")
                return
            if self.file_type.get() == "Image":
                message = decode_message_in_image(self.file_path)
            elif self.file_type.get() == "Video":
                message = decode_message_in_video(self.file_path)
            else:
                messagebox.showerror("Error", "Unsupported file type.")

            self.result_label.config(text="Decoded Message: " + message)

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2
import numpy as np
import os

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length > frame_count:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

    index = 0
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if index < message_length:
            binary_pixel = '{:08b}'.format(frame[0, 0, 0])
            frame[0, 0, 0] = int(binary_pixel[:-1] + binary_message[index], 2)
            index += 1

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    binary_message = ""
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        binary_pixel = '{:08b}'.format(frame[0, 0, 0])
        binary_message += binary_pixel[-1]

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    cap.release()
    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_or_decode = tk.StringVar()
        self.encode_or_decode.set("Encode")

        self.mode_frame = tk.Frame(self.master)
        self.mode_frame.pack()

        self.encode_radio = tk.Radiobutton(self.mode_frame, text="Encode", variable=self.encode_or_decode, value="Encode", command=self.show_encode_options)
        self.encode_radio.pack(side="left")

        self.decode_radio = tk.Radiobutton(self.mode_frame, text="Decode", variable=self.encode_or_decode, value="Decode", command=self.show_decode_options)
        self.decode_radio.pack(side="left")

        self.file_type = tk.StringVar()
        self.file_type.set("Image")

        self.encode_options_frame = tk.Frame(self.master)
        self.encode_options_frame.pack()

        self.image_radio = tk.Radiobutton(self.encode_options_frame, text="Image", variable=self.file_type, value="Image")
        self.image_radio.pack(side="left")

        self.video_radio = tk.Radiobutton(self.encode_options_frame, text="Video", variable=self.file_type, value="Video")
        self.video_radio.pack(side="left")

        self.select_button = tk.Button(self.master, text="Select File", command=self.select_file)
        self.select_button.pack()

        self.output_name_frame = tk.Frame(self.master)
        self.output_name_frame.pack()

        self.output_name_label = tk.Label(self.output_name_frame, text="Output File Name:")
        self.output_name_label.pack(side="left")

        self.output_name_entry = tk.Entry(self.output_name_frame, width=50)
        self.output_name_entry.pack(pady=5)

        self.message_frame = tk.Frame(self.master)
        self.message_frame.pack()

        self.message_label = tk.Label(self.message_frame, text="Enter Message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.message_frame, width=50)
        self.message_entry.pack(pady=5)

        self.action_button = tk.Button(self.master, textvariable=self.encode_or_decode, command=self.perform_action)
        self.action_button.pack()

        self.result_frame = tk.Frame(self.master)
        self.result_frame.pack()

        self.result_label = tk.Label(self.result_frame, text="")
        self.result_label.pack()

    def show_encode_options(self):
        if self.encode_or_decode.get() == "Encode":
            self.encode_options_frame.pack()

    def show_decode_options(self):
        if self.encode_or_decode.get() == "Decode":
            self.encode_options_frame.pack_forget()

    def select_file(self):
        file_path = filedialog.askopenfilename()
        self.file_path = file_path

    def perform_action(self):
        message = self.message_entry.get()
        output_name = self.output_name_entry.get()

        if not message:
            messagebox.showerror("Error", "Please enter a message.")
            return

        if not output_name:
            messagebox.showerror("Error", "Please enter an output file name.")
            return

        if self.encode_or_decode.get() == "Encode":
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to encode.")
                return
            if self.file_type.get() == "Image":
                output_path = os.path.splitext(self.file_path)[0] + "_" + output_name + ".png"
                hide_message_in_image(self.file_path, message, output_path)
            elif self.file_type.get() == "Video":
                output_path = os.path.splitext(self.file_path)[0] + "_" + output_name + ".mp4"
                hide_message_in_video(self.file_path, message, output_path)
            else:
                messagebox.showerror("Error", "Unsupported file type.")
        else:
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to decode.")
                return
            if self.file_type.get() == "Image":
                message = decode_message_in_image(self.file_path)
            elif self.file_type.get() == "Video":
                message = decode_message_in_video(self.file_path)
            else:
                messagebox.showerror("Error", "Unsupported file type.")

            self.result_label.config(text="Decoded Message: " + message)

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [11]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2
import numpy as np
import os

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length > frame_count:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

    index = 0
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if index < message_length:
            binary_pixel = '{:08b}'.format(frame[0, 0, 0])
            frame[0, 0, 0] = int(binary_pixel[:-1] + binary_message[index], 2)
            index += 1

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    binary_message = ""
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        binary_pixel = '{:08b}'.format(frame[0, 0, 0])
        binary_message += binary_pixel[-1]

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    cap.release()
    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_or_decode = tk.StringVar()
        self.encode_or_decode.set("Encode")

        self.mode_frame = tk.Frame(self.master)
        self.mode_frame.pack()

        self.encode_radio = tk.Radiobutton(self.mode_frame, text="Encode", variable=self.encode_or_decode, value="Encode", command=self.show_encode_options)
        self.encode_radio.pack(side="left")

        self.decode_radio = tk.Radiobutton(self.mode_frame, text="Decode", variable=self.encode_or_decode, value="Decode", command=self.show_decode_options)
        self.decode_radio.pack(side="left")

        self.file_type = tk.StringVar()
        self.file_type.set("Image")

        self.encode_options_frame = tk.Frame(self.master)
        self.encode_options_frame.pack()

        self.image_radio = tk.Radiobutton(self.encode_options_frame, text="Image", variable=self.file_type, value="Image")
        self.image_radio.pack(side="left")

        self.video_radio = tk.Radiobutton(self.encode_options_frame, text="Video", variable=self.file_type, value="Video")
        self.video_radio.pack(side="left")

        self.select_button = tk.Button(self.master, text="Select File", command=self.select_file)
        self.select_button.pack()

        self.message_frame = tk.Frame(self.master)
        self.message_frame.pack()

        self.message_label = tk.Label(self.message_frame, text="Enter Message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.message_frame, width=50)
        self.message_entry.pack(pady=5)

        self.action_button = tk.Button(self.master, textvariable=self.encode_or_decode, command=self.perform_action)
        self.action_button.pack()

        self.result_frame = tk.Frame(self.master)
        self.result_frame.pack()

        self.result_label = tk.Label(self.result_frame, text="")
        self.result_label.pack()

    def show_encode_options(self):
        if self.encode_or_decode.get() == "Encode":
            self.encode_options_frame.pack()

    def show_decode_options(self):
        if self.encode_or_decode.get() == "Decode":
            self.encode_options_frame.pack_forget()

    def select_file(self):
        file_path = filedialog.askopenfilename()
        self.file_path = file_path

    def perform_action(self):
        message = self.message_entry.get()

        if not message:
            messagebox.showerror("Error", "Please enter a message.")
            return

        if self.encode_or_decode.get() == "Encode":
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to encode.")
                return
            if self.file_type.get() == "Image":
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.png"
                hide_message_in_image(self.file_path, message, output_path)
            elif self.file_type.get() == "Video":
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.mp4"
                hide_message_in_video(self.file_path, message, output_path)
            else:
                messagebox.showerror("Error", "Unsupported file type.")
        else:
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to decode.")
                return
            if self.file_type.get() == "Image":
                message = decode_message_in_image(self.file_path)
            elif self.file_type.get() == "Video":
                message = decode_message_in_video(self.file_path)
            else:
                messagebox.showerror("Error", "Unsupported file type.")

            self.result_label.config(text="Decoded Message: " + message)

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2
import numpy as np
import os

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length > frame_count:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

    index = 0
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if index < message_length:
            binary_pixel = '{:08b}'.format(frame[0, 0, 0])
            frame[0, 0, 0] = int(binary_pixel[:-1] + binary_message[index], 2)
            index += 1

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    binary_message = ""
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        binary_pixel = '{:08b}'.format(frame[0, 0, 0])
        binary_message += binary_pixel[-1]

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    cap.release()
    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_or_decode = tk.StringVar()
        self.encode_or_decode.set("Encode")

        self.mode_frame = tk.Frame(self.master)
        self.mode_frame.pack()

        self.encode_radio = tk.Radiobutton(self.mode_frame, text="Encode", variable=self.encode_or_decode, value="Encode", command=self.show_encode_options)
        self.encode_radio.pack(side="left")

        self.decode_radio = tk.Radiobutton(self.mode_frame, text="Decode", variable=self.encode_or_decode, value="Decode", command=self.show_decode_options)
        self.decode_radio.pack(side="left")

        self.file_type = tk.StringVar()
        self.file_type.set("Image")

        self.encode_options_frame = tk.Frame(self.master)
        self.encode_options_frame.pack()

        self.image_radio = tk.Radiobutton(self.encode_options_frame, text="Image", variable=self.file_type, value="Image")
        self.image_radio.pack(side="left")

        self.video_radio = tk.Radiobutton(self.encode_options_frame, text="Video", variable=self.file_type, value="Video")
        self.video_radio.pack(side="left")

        self.select_button = tk.Button(self.master, text="Select File", command=self.select_file)
        self.select_button.pack()

        self.message_frame = tk.Frame(self.master)
        self.message_frame.pack()

        self.message_label = tk.Label(self.message_frame, text="Enter Message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.message_frame, width=50)
        self.message_entry.pack(pady=5)

        self.action_button = tk.Button(self.master, textvariable=self.encode_or_decode, command=self.perform_action)
        self.action_button.pack()

        self.result_frame = tk.Frame(self.master)
        self.result_frame.pack()

        self.result_label = tk.Label(self.result_frame, text="")
        self.result_label.pack()

    def show_encode_options(self):
        if self.encode_or_decode.get() == "Encode":
            self.encode_options_frame.pack()

    def show_decode_options(self):
        if self.encode_or_decode.get() == "Decode":
            self.encode_options_frame.pack_forget()
            file_path = filedialog.askopenfilename()
            if file_path:
                if self.file_type.get() == "Image":
                    message = decode_message_in_image(file_path)
                elif self.file_type.get() == "Video":
                    message = decode_message_in_video(file_path)
                self.result_label.config(text="Decoded Message: " + message)

    def select_file(self):
        file_path = filedialog.askopenfilename()
        self.file_path = file_path

    def perform_action(self):
        message = self.message_entry.get()

        if not message:
            messagebox.showerror("Error", "Please enter a message.")
            return

        if self.encode_or_decode.get() == "Encode":
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to encode.")
                return
            if self.file_type.get() == "Image":
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.png"
                hide_message_in_image(self.file_path, message, output_path)
            elif self.file_type.get() == "Video":
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.mp4"
                hide_message_in_video(self.file_path, message, output_path)
            else:
                messagebox.showerror("Error", "Unsupported file type.")
        else:
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to decode.")
                return
            if self.file_type.get() == "Image":
                message = decode_message_in_image(self.file_path)
            elif self.file_type.get() == "Video":
                message = decode_message_in_video(self.file_path)
            else:
                messagebox.showerror("Error", "Unsupported file type.")

            self.result_label.config(text="Decoded Message: " + message)

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


# IMG

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2
import numpy as np
import os

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length > frame_count:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

    index = 0
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if index < message_length:
            binary_pixel = '{:08b}'.format(frame[0, 0, 0])
            frame[0, 0, 0] = int(binary_pixel[:-1] + binary_message[index], 2)
            index += 1

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    binary_message = ""
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        binary_pixel = '{:08b}'.format(frame[0, 0, 0])
        binary_message += binary_pixel[-1]

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    cap.release()
    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_or_decode = tk.StringVar()
        self.encode_or_decode.set("Encode")

        self.mode_frame = tk.Frame(self.master)
        self.mode_frame.pack()

        self.encode_radio = tk.Radiobutton(self.mode_frame, text="Encode", variable=self.encode_or_decode, value="Encode")
        self.encode_radio.pack(side="left")

        self.decode_radio = tk.Radiobutton(self.mode_frame, text="Decode", variable=self.encode_or_decode, value="Decode")
        self.decode_radio.pack(side="left")

        self.select_button = tk.Button(self.master, text="Select File", command=self.select_file)
        self.select_button.pack()

        self.message_frame = tk.Frame(self.master)
        self.message_frame.pack()

        self.message_label = tk.Label(self.message_frame, text="Enter Message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.message_frame, width=50)
        self.message_entry.pack(pady=5)

        self.action_button = tk.Button(self.master, textvariable=self.encode_or_decode, command=self.perform_action)
        self.action_button.pack()

        self.result_frame = tk.Frame(self.master)
        self.result_frame.pack()

        self.result_label = tk.Label(self.result_frame, text="")
        self.result_label.pack()

    def select_file(self):
        file_path = filedialog.askopenfilename()
        self.file_path = file_path

    def perform_action(self):
        message = self.message_entry.get()

        if not message:
            messagebox.showerror("Error", "Please enter a message.")
            return

        if self.encode_or_decode.get() == "Encode":
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to encode.")
                return
            if os.path.splitext(self.file_path)[1].lower() in ['.png', '.jpg', '.jpeg']:
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.png"
                hide_message_in_image(self.file_path, message, output_path)
            elif os.path.splitext(self.file_path)[1].lower() in ['.mp4']:
                output_path = os.path.splitext(self.file_path)[0] + "_hidden.mp4"
                hide_message_in_video(self.file_path, message, output_path)
            else:
                messagebox.showerror("Error", "Unsupported file format.")
        else:
            if not self.file_path:
                messagebox.showerror("Error", "Please select a file to decode.")
                return
            if os.path.splitext(self.file_path)[1].lower() in ['.png', '.jpg', '.jpeg']:
                message = decode_message_in_image(self.file_path)
            elif os.path.splitext(self.file_path)[1].lower() in ['.mp4']:
                message = decode_message_in_video(self.file_path)
            else:
                messagebox.showerror("Error", "Unsupported file format.")

            self.result_label.config(text="Decoded Message: " + message)

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
from PIL import Image

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def hide_message_in_video(video_path, message, output_video_path):
    cap = cv2.VideoCapture(video_path)
    codec = cv2.VideoWriter_fourcc(*'XVID')
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    out = cv2.VideoWriter(output_video_path, codec, fps, frame_size)

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for i in range(frame.shape[0]):
            for j in range(frame.shape[1]):
                if index < message_length:
                    for k in range(3):
                        frame[i, j, k] = (frame[i, j, k] & 0xFE) | int(binary_message[index])
                        index += 1
                else:
                    break
            else:
                continue
            break

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_video_path))

def decode_message_from_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def decode_message_from_video(video_path):
    cap = cv2.VideoCapture(video_path)

    binary_message = ""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for i in range(frame.shape[0]):
            for j in range(frame.shape[1]):
                for k in range(3):
                    binary_message += str(frame[i, j, k] & 1)

    cap.release()

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_frame = tk.Frame(self.master)
        self.encode_frame.pack()

        self.image_label = tk.Label(self.encode_frame, text="Select an image:")
        self.image_label.pack()

        self.image_path_entry = tk.Entry(self.encode_frame, state='disabled', width=50)
        self.image_path_entry.pack(side='left', padx=5)

        self.browse_image_button = tk.Button(self.encode_frame, text="Browse Image", command=self.browse_image)
        self.browse_image_button.pack(side='left', padx=5)

        self.video_label = tk.Label(self.encode_frame, text="Select a video:")
        self.video_label.pack()

        self.video_path_entry = tk.Entry(self.encode_frame, state='disabled', width=50)
        self.video_path_entry.pack(side='left', padx=5)

        self.browse_video_button = tk.Button(self.encode_frame, text="Browse Video", command=self.browse_video)
        self.browse_video_button.pack(side='left', padx=5)

        self.message_label = tk.Label(self.encode_frame, text="Enter a message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.encode_frame, width=50)
        self.message_entry.pack(padx=5)

        self.save_label = tk.Label(self.encode_frame, text="Choose output filename:")
        self.save_label.pack()

        self.save_entry = tk.Entry(self.encode_frame, width=50)
        self.save_entry.pack(padx=5)

        self.encode_button = tk.Button(self.encode_frame, text="Hide Message", command=self.hide_message)
        self.encode_button.pack(pady=10)

        self.decode_frame = tk.Frame(self.master)
        self.decode_frame.pack()

        self.decode_image_label = tk.Label(self.decode_frame, text="Decoded Message from Image:")
        self.decode_image_label.pack()

        self.decode_image_path_entry = tk.Entry(self.decode_frame, state='disabled', width=50)
        self.decode_image_path_entry.pack(side='left', padx=5)

        self.browse_decoded_image_button = tk.Button(self.decode_frame, text="Browse Decoded Image", command=self.browse_decoded_image)
        self.browse_decoded_image_button.pack(side='left', padx=5)

        self.decode_image_button = tk.Button(self.decode_frame, text="Decode Image", command=self.decode_image)
        self.decode_image_button.pack(pady=10)

        self.decode_video_label = tk.Label(self.decode_frame, text="Decoded Message from Video:")
        self.decode_video_label.pack()

        self.decode_video_path_entry = tk.Entry(self.decode_frame, state='disabled', width=50)
        self.decode_video_path_entry.pack(side='left', padx=5)

        self.browse_decoded_video_button = tk.Button(self.decode_frame, text="Browse Decoded Video", command=self.browse_decoded_video)
        self.browse_decoded_video_button.pack(side='left', padx=5)

        self.decode_video_button = tk.Button(self.decode_frame, text="Decode Video", command=self.decode_video)
        self.decode_video_button.pack(pady=10)

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        self.image_path_entry.config(state='normal')
        self.image_path_entry.delete(0, tk.END)
        self.image_path_entry.insert(0, file_path)
        self.image_path_entry.config(state='disabled')

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
        self.video_path_entry.config(state='normal')
        self.video_path_entry.delete(0, tk.END)
        self.video_path_entry.insert(0, file_path)
        self.video_path_entry.config(state='disabled')

    def hide_message(self):
        image_path = self.image_path_entry.get()
        video_path = self.video_path_entry.get()
        message = self.message_entry.get()
        save_path = self.save_entry.get()

        if image_path:
            output_path = save_path + "_image.png"
            hide_message_in_image(image_path, message, output_path)
        if video_path:
            output_path = save_path + "_video.mp4"
            hide_message_in_video(video_path, message, output_path)

    def browse_decoded_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        self.decode_image_path_entry.config(state='normal')
        self.decode_image_path_entry.delete(0, tk.END)
        self.decode_image_path_entry.insert(0, file_path)
        self.decode_image_path_entry.config(state='disabled')

    def decode_image(self):
        image_path = self.decode_image_path_entry.get()
        if image_path:
            message = decode_message_from_image(image_path)
            messagebox.showinfo("Decoded Message from Image", message)
        else:
            messagebox.showerror("Error", "Please select an image to decode.")

    def browse_decoded_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
        self.decode_video_path_entry.config(state='normal')
        self.decode_video_path_entry.delete(0, tk.END)
        self.decode_video_path_entry.insert(0, file_path)
        self.decode_video_path_entry.config(state='disabled')

    def decode_video(self):
        video_path = self.decode_video_path_entry.get()
        if video_path:
            message = decode_message_from_video(video_path)
            messagebox.showinfo("Decoded Message from Video", message)
        else:
            messagebox.showerror("Error", "Please select a video to decode.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
from PIL import Image

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def hide_message_in_video(video_path, message, output_video_path):
    cap = cv2.VideoCapture(video_path)
    codec = cv2.VideoWriter_fourcc(*'XVID')
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    out = cv2.VideoWriter(output_video_path, codec, fps, frame_size)

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for i in range(frame.shape[0]):
            for j in range(frame.shape[1]):
                if index < message_length:
                    for k in range(3):
                        frame[i, j, k] = (frame[i, j, k] & 0xFE) | int(binary_message[index])
                        index += 1
                else:
                    break
            else:
                continue
            break

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_video_path))

def decode_message_from_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def decode_message_from_video(video_path):
    cap = cv2.VideoCapture(video_path)

    binary_message = ""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for i in range(frame.shape[0]):
            for j in range(frame.shape[1]):
                for k in range(3):
                    binary_message += str(frame[i, j, k] & 1)

    cap.release()

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        self.encode_frame = tk.Frame(self.master)
        self.encode_frame.pack()

        self.image_label = tk.Label(self.encode_frame, text="Select an image:")
        self.image_label.pack()

        self.image_path_entry = tk.Entry(self.encode_frame, state='disabled', width=50)
        self.image_path_entry.pack(side='left', padx=5)

        self.browse_image_button = tk.Button(self.encode_frame, text="Browse Image", command=self.browse_image)
        self.browse_image_button.pack(side='left', padx=5)

        self.video_label = tk.Label(self.encode_frame, text="Select a video:")
        self.video_label.pack()

        self.video_path_entry = tk.Entry(self.encode_frame, state='disabled', width=50)
        self.video_path_entry.pack(side='left', padx=5)

        self.browse_video_button = tk.Button(self.encode_frame, text="Browse Video", command=self.browse_video)
        self.browse_video_button.pack(side='left', padx=5)

        self.message_label = tk.Label(self.encode_frame, text="Enter a message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.encode_frame, width=50)
        self.message_entry.pack(padx=5)

        self.save_label = tk.Label(self.encode_frame, text="Choose output filename:")
        self.save_label.pack()

        self.save_entry = tk.Entry(self.encode_frame, width=50)
        self.save_entry.pack(padx=5)

        self.encode_button = tk.Button(self.encode_frame, text="Hide Message", command=self.hide_message)
        self.encode_button.pack(pady=10)

        self.decode_frame = tk.Frame(self.master)
        self.decode_frame.pack()

        self.decode_image_label = tk.Label(self.decode_frame, text="Decoded Message from Image:")
        self.decode_image_label.pack()

        self.decode_image_path_entry = tk.Entry(self.decode_frame, state='disabled', width=50)
        self.decode_image_path_entry.pack(side='left', padx=5)

        self.browse_decoded_image_button = tk.Button(self.decode_frame, text="Browse Decoded Image", command=self.browse_decoded_image)
        self.browse_decoded_image_button.pack(side='left', padx=5)

        self.decode_image_button = tk.Button(self.decode_frame, text="Decode Image", command=self.decode_image)
        self.decode_image_button.pack(pady=10)

        self.decode_video_label = tk.Label(self.decode_frame, text="Decoded Message from Video:")
        self.decode_video_label.pack()

        self.decode_video_path_entry = tk.Entry(self.decode_frame, state='disabled', width=50)
        self.decode_video_path_entry.pack(side='left', padx=5)

        self.browse_decoded_video_button = tk.Button(self.decode_frame, text="Browse Decoded Video", command=self.browse_decoded_video)
        self.browse_decoded_video_button.pack(side='left', padx=5)

        self.decode_video_button = tk.Button(self.decode_frame, text="Decode Video", command=self.decode_video)
        self.decode_video_button.pack(pady=10)

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        self.image_path_entry.config(state='normal')
        self.image_path_entry.delete(0, tk.END)
        self.image_path_entry.insert(0, file_path)
        self.image_path_entry.config(state='disabled')

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
        self.video_path_entry.config(state='normal')
        self.video_path_entry.delete(0, tk.END)
        self.video_path_entry.insert(0, file_path)
        self.video_path_entry.config(state='disabled')

    def hide_message(self):
        image_path = self.image_path_entry.get()
        video_path = self.video_path_entry.get()
        message = self.message_entry.get()
        save_path = self.save_entry.get()

        if image_path:
            output_path = save_path + "_image.png"
            hide_message_in_image(image_path, message, output_path)
        if video_path:
            output_path = save_path + "_video.mp4"
            hide_message_in_video(video_path, message, output_path)

    def browse_decoded_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        self.decode_image_path_entry.config(state='normal')
        self.decode_image_path_entry.delete(0, tk.END)
        self.decode_image_path_entry.insert(0, file_path)
        self.decode_image_path_entry.config(state='disabled')

    def decode_image(self):
        image_path = self.decode_image_path_entry.get()
        if image_path:
            message = decode_message_from_image(image_path)
            messagebox.showinfo("Decoded Message from Image", message)
        else:
            messagebox.showerror("Error", "Please select an image to decode.")

    def browse_decoded_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
        self.decode_video_path_entry.config(state='normal')
        self.decode_video_path_entry.delete(0, tk.END)
        self.decode_video_path_entry.insert(0, file_path)
        self.decode_video_path_entry.config(state='disabled')

    def decode_video(self):
        video_path = self.decode_video_path_entry.get()
        if video_path:
            message = decode_message_from_video(video_path)
            messagebox.showinfo("Decoded Message from Video", message)
        else:
            messagebox.showerror("Error", "Please select a video to decode.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2
import numpy as np

def hide_message_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    frame_index = 0
    char_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for i in range(frame_height):
            for j in range(frame_width):
                if char_index < message_length:
                    pixel = frame[i, j]
                    frame[i, j] = tuple(pixel & 0xFE | int(binary_message[char_index]))
                    char_index += 1

        out.write(frame)
        frame_index += 1

    cap.release()
    out.release()

    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    decoded_message = ""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for i in range(frame.shape[0]):
            for j in range(frame.shape[1]):
                pixel = frame[i, j]
                decoded_message += str(pixel[-1] & 1)

    # Convert binary message to characters
    message = ""
    for i in range(0, len(decoded_message), 8):
        byte = decoded_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Steganography App")

        # Encoding section
        self.encode_frame = tk.Frame(self.master)
        self.encode_frame.pack()

        self.file_label = tk.Label(self.encode_frame, text="Select a file:")
        self.file_label.pack()

        self.file_path_entry = tk.Entry(self.encode_frame, state='disabled', width=50)
        self.file_path_entry.pack(side='left', padx=5)

        self.browse_button = tk.Button(self.encode_frame, text="Browse", command=self.browse_file)
        self.browse_button.pack(side='left', padx=5)

        self.message_label = tk.Label(self.encode_frame, text="Enter a message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.encode_frame, width=50)
        self.message_entry.pack(padx=5)

        self.save_label = tk.Label(self.encode_frame, text="Choose output filename:")
        self.save_label.pack()

        self.save_entry = tk.Entry(self.encode_frame, width=50)
        self.save_entry.pack(padx=5)

        self.hide_button = tk.Button(self.encode_frame, text="Hide Message", command=self.hide_message)
        self.hide_button.pack(pady=10)

        # Decoding section
        self.decode_frame = tk.Frame(self.master)
        self.decode_frame.pack()

        self.decode_label = tk.Label(self.decode_frame, text="Decode Hidden Message:")
        self.decode_label.pack()

        self.decode_file_path_entry = tk.Entry(self.decode_frame, state='disabled', width=50)
        self.decode_file_path_entry.pack(side='left', padx=5)

        self.decode_browse_button = tk.Button(self.decode_frame, text="Browse", command=self.browse_encoded_file)
        self.decode_browse_button.pack(side='left', padx=5)

        self.decode_button = tk.Button(self.decode_frame, text="Decode", command=self.decode_message)
        self.decode_button.pack(pady=10)

    def browse_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("All Files", "*.*")])
        self.file_path_entry.config(state='normal')
        self.file_path_entry.delete(0, tk.END)
        self.file_path_entry.insert(0, file_path)
        self.file_path_entry.config(state='disabled')

    def hide_message(self):
        file_path = self.file_path_entry.get()
        message = self.message_entry.get()
        save_path = self.save_entry.get()

        if file_path and message and save_path:
            output_path = save_path + ".png" if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')) else save_path + ".mp4"
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                hide_message_image(file_path, message, output_path)
            elif file_path.lower().endswith('.mp4'):
                hide_message_video(file_path, message, output_path)
        else:
            messagebox.showerror("Error", "Please select a file, enter a message, and choose an output filename.")

    def browse_encoded_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("All Files", "*.*")])
        self.decode_file_path_entry.config(state='normal')
        self.decode_file_path_entry.delete(0, tk.END)
        self.decode_file_path_entry.insert(0, file_path)
        self.decode_file_path_entry.config(state='disabled')

    def decode_message(self):
        file_path = self.decode_file_path_entry.get()
        if file_path:
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                message = decode_message_image(file_path)
            elif file_path.lower().endswith('.mp4'):
                message = decode_message_video(file_path)
            messagebox.showinfo("Decoded Message", message)
        else:
            messagebox.showerror("Error", "Please select a file to decode.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    codec = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, codec, fps, (width, height))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for y in range(height):
            for x in range(width):
                if index < message_length * 3:
                    b, g, r = frame[y, x]
                    # Hide message in LSB of each channel
                    r = (r & 0xFE) | int(binary_message[index])
                    index += 1
                    if index < message_length * 3:
                        g = (g & 0xFE) | int(binary_message[index])
                        index += 1
                    if index < message_length * 3:
                        b = (b & 0xFE) | int(binary_message[index])
                        index += 1
                    frame[y, x] = [b, g, r]
                else:
                    break

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Video Steganography App")

        self.frame = tk.Frame(self.master)
        self.frame.pack()

        self.video_label = tk.Label(self.frame, text="Select a video:")
        self.video_label.pack()

        self.video_path_entry = tk.Entry(self.frame, state='disabled', width=50)
        self.video_path_entry.pack(side='left', padx=5)

        self.video_browse_button = tk.Button(self.frame, text="Browse", command=self.browse_video)
        self.video_browse_button.pack(side='left', padx=5)

        self.message_label = tk.Label(self.frame, text="Enter a message:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.frame, width=50)
        self.message_entry.pack(padx=5)

        self.output_label = tk.Label(self.frame, text="Choose output filename:")
        self.output_label.pack()

        self.output_entry = tk.Entry(self.frame, width=50)
        self.output_entry.pack(padx=5)

        self.hide_button = tk.Button(self.frame, text="Hide Message", command=self.hide_message)
        self.hide_button.pack(pady=10)

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        self.video_path_entry.config(state='normal')
        self.video_path_entry.delete(0, tk.END)
        self.video_path_entry.insert(0, file_path)
        self.video_path_entry.config(state='disabled')

    def hide_message(self):
        video_path = self.video_path_entry.get()
        message = self.message_entry.get()
        output_path = self.output_entry.get()

        if video_path and message and output_path:
            hide_message_in_video(video_path, message, output_path)
        else:
            messagebox.showerror("Error", "Please select a video, enter a message, and choose an output filename.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    codec = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, codec, fps, (width, height))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for y in range(height):
            for x in range(width):
                if index < message_length:
                    b, g, r = frame[y, x]
                    # Hide message in LSB of each channel
                    r = (r & 0xFE) | int(binary_message[index])
                    index += 1
                    if index < message_length:
                        g = (g & 0xFE) | int(binary_message[index])
                        index += 1
                    if index < message_length:
                        b = (b & 0xFE) | int(binary_message[index])
                        index += 1
                    frame[y, x] = [b, g, r]
                else:
                    break

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    binary_message = ""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for y in range(frame.shape[0]):
            for x in range(frame.shape[1]):
                b, g, r = frame[y, x]
                binary_message += str(r & 1)
                binary_message += str(g & 1)
                binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Media Steganography App")

        # Image encoding section
        self.image_encode_frame = tk.Frame(self.master)
        self.image_encode_frame.pack()

        self.image_label = tk.Label(self.image_encode_frame, text="Select an image:")
        self.image_label.pack()

        self.image_path_entry = tk.Entry(self.image_encode_frame, state='disabled', width=50)
        self.image_path_entry.pack(side='left', padx=5)

        self.image_browse_button = tk.Button(self.image_encode_frame, text="Browse", command=self.browse_image)
        self.image_browse_button.pack(side='left', padx=5)

        self.image_message_label = tk.Label(self.image_encode_frame, text="Enter a message:")
        self.image_message_label.pack()

        self.image_message_entry = tk.Entry(self.image_encode_frame, width=50)
        self.image_message_entry.pack(padx=5)

        self.image_save_label = tk.Label(self.image_encode_frame, text="Choose output filename:")
        self.image_save_label.pack()

        self.image_save_entry = tk.Entry(self.image_encode_frame, width=50)
        self.image_save_entry.pack(padx=5)

        self.image_hide_button = tk.Button(self.image_encode_frame, text="Hide Message in Image", command=self.hide_message_in_image)
        self.image_hide_button.pack(pady=10)

        # Image decoding section
        self.image_decode_frame = tk.Frame(self.master)
        self.image_decode_frame.pack()

        self.image_decode_label = tk.Label(self.image_decode_frame, text="Decode Hidden Message from Image:")
        self.image_decode_label.pack()

        self.image_decode_path_entry = tk.Entry(self.image_decode_frame, state='disabled', width=50)
        self.image_decode_path_entry.pack(side='left', padx=5)

        self.image_decode_browse_button = tk.Button(self.image_decode_frame, text="Browse", command=self.browse_encoded_image)
        self.image_decode_browse_button.pack(side='left', padx=5)

        self.image_decode_button = tk.Button(self.image_decode_frame, text="Decode Image", command=self.decode_image)
        self.image_decode_button.pack(pady=10)

        # Video encoding section
        self.video_encode_frame = tk.Frame(self.master)
        self.video_encode_frame.pack()

        self.video_label = tk.Label(self.video_encode_frame, text="Select a video:")
        self.video_label.pack()

        self.video_path_entry = tk.Entry(self.video_encode_frame, state='disabled', width=50)
        self.video_path_entry.pack(side='left', padx=5)

        self.video_browse_button = tk.Button(self.video_encode_frame, text="Browse", command=self.browse_video)
        self.video_browse_button.pack(side='left', padx=5)

        self.video_message_label = tk.Label(self.video_encode_frame, text="Enter a message:")
        self.video_message_label.pack()

        self.video_message_entry = tk.Entry(self.video_encode_frame, width=50)
        self.video_message_entry.pack(padx=5)

        self.video_save_label = tk.Label(self.video_encode_frame, text="Choose output filename:")
        self.video_save_label.pack()

        self.video_save_entry = tk.Entry(self.video_encode_frame, width=50)
        self.video_save_entry.pack(padx=5)

        self.video_hide_button = tk.Button(self.video_encode_frame, text="Hide Message in Video", command=self.hide_message_in_video)
        self.video_hide_button.pack(pady=10)

        # Video decoding section
        self.video_decode_frame = tk.Frame(self.master)
        self.video_decode_frame.pack()

        self.video_decode_label = tk.Label(self.video_decode_frame, text="Decode Hidden Message from Video:")
        self.video_decode_label.pack()

        self.video_decode_path_entry = tk.Entry(self.video_decode_frame, state='disabled', width=50)
        self.video_decode_path_entry.pack(side='left', padx=5)

        self.video_decode_browse_button = tk.Button(self.video_decode_frame, text="Browse", command=self.browse_encoded_video)
        self.video_decode_browse_button.pack(side='left', padx=5)

        self.video_decode_button = tk.Button(self.video_decode_frame, text="Decode Video", command=self.decode_video)
        self.video_decode_button.pack(pady=10)

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        self.image_path_entry.config(state='normal')
        self.image_path_entry.delete(0, tk.END)
        self.image_path_entry.insert(0, file_path)
        self.image_path_entry.config(state='disabled')

    def hide_message_in_image(self):
        image_path = self.image_path_entry.get()
        message = self.image_message_entry.get()
        save_path = self.image_save_entry.get()

        if image_path and message and save_path:
            output_path = save_path + ".png"
            hide_message_in_image(image_path, message, output_path)
        else:
            messagebox.showerror("Error", "Please select an image, enter a message, and choose an output filename.")

    def browse_encoded_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        self.image_decode_path_entry.config(state='normal')
        self.image_decode_path_entry.delete(0, tk.END)
        self.image_decode_path_entry.insert(0, file_path)
        self.image_decode_path_entry.config(state='disabled')

    def decode_image(self):
        image_path = self.image_decode_path_entry.get()
        if image_path:
            message = decode_message_in_image(image_path)
            messagebox.showinfo("Decoded Message from Image", message)
        else:
            messagebox.showerror("Error", "Please select an encoded image to decode.")

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        self.video_path_entry.config(state='normal')
        self.video_path_entry.delete(0, tk.END)
        self.video_path_entry.insert(0, file_path)
        self.video_path_entry.config(state='disabled')

    def hide_message_in_video(self):
        video_path = self.video_path_entry.get()
        message = self.video_message_entry.get()
        save_path = self.video_save_entry.get()

        if video_path and message and save_path:
            output_path = save_path + ".mp4"
            hide_message_in_video(video_path, message, output_path)
        else:
            messagebox.showerror("Error", "Please select a video, enter a message, and choose an output filename.")

    def browse_encoded_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        self.video_decode_path_entry.config(state='normal')
        self.video_decode_path_entry.delete(0, tk.END)
        self.video_decode_path_entry.insert(0, file_path)
        self.video_decode_path_entry.config(state='disabled')

    def decode_video(self):
        video_path = self.video_decode_path_entry.get()
        if video_path:
            message = decode_message_in_video(video_path)
            messagebox.showinfo("Decoded Message from Video", message)
        else:
            messagebox.showerror("Error", "Please select an encoded video to decode.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    codec = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, codec, fps, (width, height))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for y in range(height):
            for x in range(width):
                if index < message_length:
                    b, g, r = frame[y, x]
                    # Hide message in LSB of each channel
                    r = (r & 0xFE) | int(binary_message[index])
                    index += 1
                    if index < message_length:
                        g = (g & 0xFE) | int(binary_message[index])
                        index += 1
                    if index < message_length:
                        b = (b & 0xFE) | int(binary_message[index])
                        index += 1
                    frame[y, x] = [b, g, r]
                else:
                    break

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    binary_message = ""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for y in range(frame.shape[0]):
            for x in range(frame.shape[1]):
                b, g, r = frame[y, x]
                binary_message += str(r & 1)
                binary_message += str(g & 1)
                binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Media Steganography App")

        # Video encoding section
        self.video_encode_frame = tk.Frame(self.master)
        self.video_encode_frame.pack()

        self.video_label = tk.Label(self.video_encode_frame, text="Select a video:")
        self.video_label.pack()

        self.video_path_entry = tk.Entry(self.video_encode_frame, state='disabled', width=50)
        self.video_path_entry.pack(side='left', padx=5)

        self.video_browse_button = tk.Button(self.video_encode_frame, text="Browse", command=self.browse_video)
        self.video_browse_button.pack(side='left', padx=5)

        self.video_message_label = tk.Label(self.video_encode_frame, text="Enter a message:")
        self.video_message_label.pack()

        self.video_message_entry = tk.Entry(self.video_encode_frame, width=50)
        self.video_message_entry.pack(padx=5)

        self.video_save_label = tk.Label(self.video_encode_frame, text="Choose output filename:")
        self.video_save_label.pack()

        self.video_save_entry = tk.Entry(self.video_encode_frame, width=50)
        self.video_save_entry.pack(padx=5)

        self.video_hide_button = tk.Button(self.video_encode_frame, text="Hide Message in Video", command=self.hide_message_in_video)
        self.video_hide_button.pack(pady=10)

        # Video decoding section
        self.video_decode_frame = tk.Frame(self.master)
        self.video_decode_frame.pack()

        self.video_decode_label = tk.Label(self.video_decode_frame, text="Decode Hidden Message from Video:")
        self.video_decode_label.pack()

        self.video_decode_path_entry = tk.Entry(self.video_decode_frame, state='disabled', width=50)
        self.video_decode_path_entry.pack(side='left', padx=5)

        self.video_decode_browse_button = tk.Button(self.video_decode_frame, text="Browse", command=self.browse_encoded_video)
        self.video_decode_browse_button.pack(side='left', padx=5)

        self.video_decode_button = tk.Button(self.video_decode_frame, text="Decode Video", command=self.decode_video)
        self.video_decode_button.pack(pady=10)

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        self.video_path_entry.config(state='normal')
        self.video_path_entry.delete(0, tk.END)
        self.video_path_entry.insert(0, file_path)
        self.video_path_entry.config(state='disabled')

    def hide_message_in_video(self):
        video_path = self.video_path_entry.get()
        message = self.video_message_entry.get()
        save_path = self.video_save_entry.get()

        if video_path and message and save_path:
            output_path = save_path + ".mp4"
            hide_message_in_video(video_path, message, output_path)
        else:
            messagebox.showerror("Error", "Please select a video, enter a message, and choose an output filename.")

    def browse_encoded_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        self.video_decode_path_entry.config(state='normal')
        self.video_decode_path_entry.delete(0, tk.END)
        self.video_decode_path_entry.insert(0, file_path)
        self.video_decode_path_entry.config(state='disabled')

    def decode_video(self):
        video_path = self.video_decode_path_entry.get()
        if video_path:
            message = decode_message_in_video(video_path)
            messagebox.showinfo("Decoded Message from Video", message)
        else:
            messagebox.showerror("Error", "Please select an encoded video to decode.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()
